In [4]:
# CS440_MP01_V1
# Team Member:Zhenwei Wu, Yuji Chai, Zhoushi Zhu
# function list
# drawMaze(ls): print out maze;
# heuristic(ls,end): generate distance from target(or 'food'), return a 2D list
# drawSol(ls,map): draw solution on maze(or 'origin_mazeList')
# bfs(ls, start), dfs(ls, start), A_star(ls),
# 1) open text file by fp; 
# 2) copy text file into mazeList[], use in path searching; 
# 3) deepcopy origin_mazeList[] from mazeList[], use in drawSol(ls,map).
fp = './tinySearch.txt'
mazeList=[]
origin_mazeList=[]
mazeMapList=[]
sortedFood = []
row=0
column=0
ct=0
pecman=0
foodList = []
foodSol = []
TOTAL = 0
foodMap = {}
Edge = []

H = []
import numpy as np
import copy
from copy import deepcopy

with open(fp) as f:  
    firstLine = f.readline()
    temp = []
    for a in firstLine:
        if a == '%':
            column+=1
            temp.append('%')
            ct+=1
    mazeList.append(temp)
    row+=1
    
    for line in f:
        row+=1
        temp=[]
        for a in line:
            if(a=='%'):
                temp.append('%')
                ct+=1
            elif(a=='P'):
                temp.append('P')
                ct+=1
                pecman=ct
            elif(a=='.'):
                temp.append('.')
                ct+=1
                foodList.append(ct)
                TOTAL+=1
                
            elif(a==' '):
                temp.append(' ')
                ct+=1
        mazeList.append(temp)
        

origin_mazeList=copy.deepcopy(mazeList)
foodList.insert(0,pecman)

def heuristic(start,end):#manhattan
    
    x1=int(end/column)
    y1=int(end%column-1)
    x2=int(start/column)
    y2=int(start%column-1)
    
    return abs(x1-x2)+abs(y1-y2)


def drawMaze(ls):
    for i in ls:
        temp=""
        for j in i:
            temp+=j
        print(temp)
        
def A_star_cost(ls,start,end):
    stack=[[start,heuristic(start,end),0]]
    while stack:
        elem=stack.pop(0)
        ind=elem[0]
        #calculate row as 'r', column as 'c', find current content use ls[r][c]
        r=int(ind/column)
        c=ind%column-1
        cur=ls[r][c]
        if cur != '%' and cur != 'V':
            if ind == end:
                return elem[2]
            #print(r,',',c,ls[r][c])
            ls[r][c]='V'
            
            # down
            if (r+1)<row:
                if ls[r+1][c] != 'V' and ls[r+1][c] != '%':
                    temp=ind+column
#                     mazeMap[temp]=ind
                    #print(r+1,',',c,ls[r+1][c])
                    stack.append([temp,heuristic(ind+column, end)+elem[2]+1,elem[2]+1])
            # up
            if (r-1)>=0:
                if ls[r-1][c] != 'V' and ls[r-1][c] != '%':
                    temp=ind-column
#                     mazeMap[temp]=ind
                    #print(r-1,',',c,ls[r-1][c])
                    stack.append([temp,heuristic(ind-column, end)+elem[2]+1,elem[2]+1])
            # right
            if (c+1)<column:
                if ls[r][c+1] != 'V' and ls[r][c+1] != '%':
                    temp=ind+1                  
#                   mazeMap[temp]=ind
                    #print(r,',',c+1,ls[r][c+1])
                    stack.append([temp,heuristic(ind+1, end)+elem[2]+1,elem[2]+1])
            # left
            if (c-1)>=0:
                if ls[r][c-1] != 'V' and ls[r][c-1] != '%':
                    temp=ind-1       
#                     mazeMap[temp]=ind
                    #print(r,',',c-1,ls[r][c-1])
                    stack.append([temp,heuristic(ind+1, end)+elem[2]+1,elem[2]+1])
            # sort stack by distance
            stack=sorted(stack, key=lambda a: a[1])
        
        
drawMaze(mazeList)
print("row: ",row," ;column: ",column)
print("pecman: ",pecman," ;food: ",TOTAL)

ASTAR = {}

for i in range(len(foodList)):
    temp1 = {}
    for j in range(len(foodList)):
        temp2 = 0 if i==j else A_star_cost(deepcopy(mazeList), foodList[i], foodList[j])
        temp1[foodList[j]] = temp2
    ASTAR[foodList[i]] = deepcopy(temp1)
    
del foodList[0]
        


def addelement_dset(dset, num):
    for i in range(num):
        dset.append(-1)

def find_dset(dset, elem):
    if(dset[elem] < 0):
        return elem
    else:
        find_result = find_dset(dset,dset[elem])
        dset[elem] = find_result
        return find_result

def setunion_dset(dset, a, b):
    root1 = find_dset(dset,a)
    root2 = find_dset(dset,b)
    
    newsize = dset[root1] + dset[root2]
    if(dset[root1] >= dset[root2]):
        dset[root2] = root1
        dset[root1] = newsize
    else:
        dset[root1] = root2
        dset[root2] = newsize

def prim(foodlist):
    dset = []
    Edge = hForAll(foodlist)
    
    addelement_dset(dset, len(foodlist))#aasdasdasdasdasdasdasdasd
    Edge = sorted(Edge, key = lambda x : x[2])
#     print(Edge)
    edge_count = 0
    total_dist = 0
    
    for it in Edge:
        if(find_dset(dset,it[0]) != find_dset(dset,it[1])):
            total_dist += it[2]
            setunion_dset(dset,it[0], it[1])
    return total_dist
    
def sort_food(foodlist, start, total):
    dist_list = [[0, 0, 0]]#[current dist, prim_mst]
    visited_list = {}#current visited
    remaining_list = {}#remaining food
    list_idx = 0
    res_list = []
    
    # init of dist_list variables
    dl_curr = 0
    dl_child = 0
    dl_parent = 0
    dl_temp = [0, 0, 0]
    
    for i in range(len(foodlist)):#initializing containers with first step
        templist = deepcopy(foodlist)
        curr_visit = templist[i]
#         print(curr_visit)
        del templist[i]
        
        temp_dist = []
        temp_visited = []
        
        temp_dist.append(h_Astar(start, curr_visit))
        temp_dist.append(prim(templist))
        temp_dist.append(list_idx)
        
        temp_visited.append(curr_visit)
        
        # Add element start
        dist_list.append(temp_dist)
        dl_curr = len(dist_list) - 1

        while(dl_curr > 1):
            dl_parent = int(dl_curr / 2)
            if(dist_list[dl_parent][0] + dist_list[dl_parent][1] <= dist_list[dl_curr][0] + dist_list[dl_curr][1]):
                break
            else:
                dl_temp = dist_list[dl_parent]
                dist_list[dl_parent] = dist_list[dl_curr]
                dist_list[dl_curr] = dl_temp
                dl_curr = dl_parent
        # Add element end      
        
        visited_list[list_idx] = temp_visited
        remaining_list[list_idx] = templist
        list_idx += 1
        
    print("idx ",list_idx)
        
    while 1:
        min_dist = deepcopy(dist_list[1])
        min_idx = min_dist[2]
        min_visited_list = deepcopy(visited_list[min_idx])
        # print(min_dist[0]+min_dist[1])
        min_remaining_list = deepcopy(remaining_list[min_idx])
        
        # Remove elements start
        dl_curr = 1

        dl_temp = dist_list[-1]
        dist_list[-1] = dist_list[dl_curr]
        dist_list[dl_curr] = dl_temp
        # print(dist_list)
        dist_list.pop() # remove the last element

        while(dl_curr <= int((len(dist_list) - 1) / 2)):
            dl_child = dl_curr * 2
            if(dl_child + 1 < len(dist_list)):
                if(dist_list[dl_child][0] + dist_list[dl_child][1] > dist_list[dl_child + 1][0] + dist_list[dl_child + 1][1]):
                    dl_child += 1

            if(dist_list[dl_child][0] + dist_list[dl_child][1] >= dist_list[dl_curr][0] + dist_list[dl_curr][1]):
                break
            else:
                dl_temp = dist_list[dl_child]
                dist_list[dl_child] = dist_list[dl_curr]
                dist_list[dl_curr] = dl_temp
                dl_curr = dl_child
        # Remove element end
        
        del visited_list[min_idx]
        del remaining_list[min_idx]
        
        if len(min_visited_list) == total:
            res_list = deepcopy(min_visited_list)
            print("shortest man: ", min_dist[0], min_dist[1])
            
            break
         
        for i in range(len(min_remaining_list)):
            templist = deepcopy(min_remaining_list)
            curr_visit = templist[i]
            temp_visited = deepcopy(min_visited_list)
            
            del templist[i]

            temp_dist = []
            
            temp_dist.append(h_Astar(min_visited_list[-1], curr_visit) + min_dist[0])
            temp_dist.append(prim(templist))
            temp_dist.append(list_idx)

            temp_visited.append(curr_visit)

            # Add element start
            dist_list.append(temp_dist)
            dl_curr = len(dist_list) - 1

            while(dl_curr > 1):
                dl_parent = int(dl_curr / 2)
                if(dist_list[dl_parent][0] + dist_list[dl_parent][1] <= dist_list[dl_curr][0] + dist_list[dl_curr][1]):
                    break
                else:
                    dl_temp = dist_list[dl_parent]
                    dist_list[dl_parent] = dist_list[dl_curr]
                    dist_list[dl_curr] = dl_temp
                    dl_curr = dl_parent
            # Add element end 
            
            visited_list[list_idx] = (temp_visited)
            remaining_list[list_idx] = (templist)
            list_idx += 1
        
        
    return res_list


def h_Astar(start,end):
    return ASTAR[start][end]

def hForAll(foodList):
    h_for_all = []
    for i in range(len(foodList)):
        for j in range(i):
            if foodList[i] != foodList[j]:
                temp = [i,j,h_Astar(foodList[i],foodList[j])]
                h_for_all.append(temp)
    return h_for_all


def A_star_multiple(ls, foodlist, start, total):
    backup = deepcopy(ls)
    mazeMap={}
    dist = 0
    
    sorted_list = sort_food(foodlist, start, total)
    print("sorted list   :", sorted_list)
    sortedFood = sorted_list
    next_target = deepcopy(sorted_list[0])
        
    stack=[[start,heuristic(start, next_target),0]]

    while stack:
        elem=stack.pop(0)
        ind=elem[0]
        #calculate row as 'r', column as 'c', find current content use ls[r][c]
        r=int(ind/column)
        c=ind%column-1
        cur=ls[r][c]
#         print('ind = ', ind)
#         print('cur = ', cur)
#         print('next = ', next_target)
#         print('r = ', r)
#         print('c = ', c)
        if cur != '%' and cur != 'V':
            if cur == '.':
                if ind != next_target:
                    print("remove:", ind)
                    print("currlist:", sorted_list)
                    sorted_list.remove(ind)
                    backup[r][c]=' '
                    dist+=elem[2]
                    sorted_list = sort_food(sorted_list, ind, len(sorted_list))
                    mazeMapList.append(deepcopy(mazeMap))
                    mazeMap.clear()
                    ls = deepcopy(backup)

                    curr_start = ind
                    if len(sorted_list) == 0:
                        break

                    next_target = deepcopy(sorted_list[0])
                    stack=[[curr_start,heuristic(curr_start, next_target),0]]
                    continue
                    
                else:
                    print('a')
                    print("remove:", ind)
                    print("currlist:", sorted_list)
                    sorted_list.remove(ind)
                    backup[r][c]=' '
                    dist += elem[2]
                    mazeMapList.append(deepcopy(mazeMap))
                    mazeMap.clear()
                    ls = deepcopy(backup)

                    curr_start = next_target
                    if len(sorted_list) == 0:
                        break

                    next_target = deepcopy(sorted_list[0])
                    
                    
                    stack=[[curr_start,heuristic(curr_start, next_target),0]]
                    continue
                    
            #print(r,',',c,ls[r][c])
            ls[r][c]='V'
            
            # down
            if (r+1)<row:
                if ls[r+1][c] != 'V' and ls[r+1][c] != '%':
                    temp=ind+column
                    mazeMap[temp]=ind
                    #print(r+1,',',c,ls[r+1][c])
                    stack.append([temp,heuristic(ind+column, next_target)+elem[2]+1,elem[2]+1])
            # up
            if (r-1)>=0:
                if ls[r-1][c] != 'V' and ls[r-1][c] != '%':
                    temp=ind-column
                    mazeMap[temp]=ind
                    #print(r-1,',',c,ls[r-1][c])
                    stack.append([temp,heuristic(ind-column, next_target)+elem[2]+1,elem[2]+1])
            # right
            if (c+1)<column:
                if ls[r][c+1] != 'V' and ls[r][c+1] != '%':
                    temp=ind+1                  
                    mazeMap[temp]=ind
                    #print(r,',',c+1,ls[r][c+1])
                    stack.append([temp,heuristic(ind+1, next_target)+elem[2]+1,elem[2]+1])
            # left
            if (c-1)>=0:
                if ls[r][c-1] != 'V' and ls[r][c-1] != '%':
                    temp=ind-1       
                    mazeMap[temp]=ind
                    #print(r,',',c-1,ls[r][c-1])
                    stack.append([temp,heuristic(ind-1, next_target)+elem[2]+1,elem[2]+1])
            # sort stack by distance
            stack=sorted(stack, key=lambda a: a[1])
    
    return dist
    
    
# draw solution from food->pecman on mazeList; 
# need drawMaze(mazeList) to visualize path 
# ct: total steps
       
def drawSol(ls, start, end, solutionMap):
    x=end
    ct=0
    while x != start:
        #print(x)
        r=int(x/column)
        c=x%column-1
        ls[r][c]='*'
        x=solutionMap[x]
        ct+=1
    return ct

def drawSols(ls, foodList, solutionList):
    print(foodList)
    print(solutionList)
    
    count = 0
    temp_count = 0
    for i in range(len(solutionList)):
        temp = copy.deepcopy(ls)
        temp_count = drawSol(temp, foodList[i], foodList[i + 1], solutionList[i])
        count += temp_count
        print("The solution for the ", i+1, "round:")
        print("This round costs ", temp_count, "steps.")
        drawMaze(temp)
        print("")
    print("The entire solution costs " , count , "steps.")
    return count




%%%%%%%%%%
%.  %   .%
% %.% %% %
% %   .%.%
% .%P%   %
%.  .  . %
% %%%% %.%
%.    .% %
%%%%%%%%%%
row:  9  ;column:  10
pecman:  45  ;food:  12


In [6]:
import time

start = time.time()
sort_food(foodList, pecman, TOTAL)
end = time.time()
print(end - start)


idx  12
shortest man:  35 0
0.4025750160217285


In [5]:
A_star_multiple(mazeList,foodList,pecman, TOTAL)

idx  12
shortest man:  35 0
sorted list   : [55, 52, 72, 43, 12, 24, 37, 77, 58, 69, 39, 19]
a
remove: 55
currlist: [55, 52, 72, 43, 12, 24, 37, 77, 58, 69, 39, 19]
a
remove: 52
currlist: [52, 72, 43, 12, 24, 37, 77, 58, 69, 39, 19]
a
remove: 72
currlist: [72, 43, 12, 24, 37, 77, 58, 69, 39, 19]
a
remove: 43
currlist: [43, 12, 24, 37, 77, 58, 69, 39, 19]
a
remove: 12
currlist: [12, 24, 37, 77, 58, 69, 39, 19]
a
remove: 24
currlist: [24, 37, 77, 58, 69, 39, 19]
a
remove: 37
currlist: [37, 77, 58, 69, 39, 19]
a
remove: 77
currlist: [77, 58, 69, 39, 19]
a
remove: 58
currlist: [58, 69, 39, 19]
a
remove: 69
currlist: [69, 39, 19]
a
remove: 39
currlist: [39, 19]
a
remove: 19
currlist: [19]


35

In [155]:

drawSols(origin_mazeList ,foodSol, mazeMapList)

[]
[{55: 45, 35: 45}, {45: 55, 56: 55, 54: 55, 35: 45, 53: 54, 43: 53, 52: 53}, {53: 43, 42: 43, 54: 53, 52: 42, 32: 42}, {62: 52, 42: 52, 53: 52, 72: 62}, {62: 72, 73: 72, 52: 62, 42: 52, 53: 52, 32: 42, 43: 42, 22: 32, 12: 22}, {22: 12, 13: 12, 32: 22, 14: 13, 24: 14}, {34: 24, 14: 24, 35: 34, 45: 35, 36: 35, 26: 36, 37: 36}, {47: 37, 36: 37, 57: 47, 48: 47, 67: 57, 58: 57, 56: 57, 77: 67}, {67: 77, 76: 77, 57: 67, 47: 57, 58: 57, 56: 57}, {48: 58, 59: 58, 57: 58, 69: 59, 49: 59}, {79: 69, 59: 69, 49: 59, 58: 59, 39: 49, 48: 49}, {49: 39, 29: 39, 19: 29}]


IndexError: list index out of range